## Parte 1 - Recolha de tweets com cashtag $AAPL 


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas 

from emoji import UNICODE_EMOJI
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
from collections import Counter

# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def is_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI['en']:
        count += s.count(emoji)
    return count

def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))


################# tweets samples #########################
nltk.download('twitter_samples')
                
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

###################### POSITIVE WORDS ####################

list_pos = []
for x in all_positive_tweets:
    x = " ".join(filter(lambda x:x[0]!='@', x.split()))
    for i in x.split():
        list_pos.append(i)
##### Extra https://ptrckprry.com/course/ssd/data/positive-words.txt ##
with open('pos_words.txt', 'r') as f:
    myNames = [line.strip() for line in f]

list_pos.extend(myNames)
print(len(list_pos))
#########################################################
##################### NEGATIVE WORDS ###################
list_neg = []
for x in all_negative_tweets:
    x = " ".join(filter(lambda x:x[0]!='@', x.split()))
    for i in x.split():
        list_neg.append(i)
##### Extra https://ptrckprry.com/course/ssd/data/negative-words.txt ##
with open('neg_words.txt', 'r') as f:
    myNames = [line.strip() for line in f]

list_neg.extend(myNames)
print(len(list_neg))
########################################################
# Creating list to append tweet data to
tweets_list = []
def get_tweets():
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper('$AAPL until:2022-02-12 since:2022-01-28 lang:en').get_items()):
        if "$AAPL" in tweet.content: 
           #remove urls
            regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
            url = re.findall(regex,tweet.content)
            #remove emojis
            #print(is_emoji(tweet.content))
            #remove urls and emojis
            if url == [] and is_emoji(tweet.content) == 0:

                ####################### Pré-processamento ##################################
                # remove special characters
                tweet.content = " ".join(filter(lambda x:x[0]!='$', tweet.content.split()))
                tweet.content = re.sub(r"[^a-zA-Z0-9]+", ' ', tweet.content)
                tweet.content.lower()

                #remove \n and \t
                tweet_words_list = tweet.content.split(" ")
                tweet_new_list = []
                for word in tweet_words_list:
                    list_word = word.split("\n")
                    word = listToString(list_word)
                    word = word.strip('\n')
                    word = word.strip('\t')
                    tweet_new_list.append(word)
                tweet.content = listToString(tweet_new_list)

                ######################## Número de palavras ################################
                pos_count_tweet = 0
                neg_count_tweet = 0
                total_count_tweet = 0
                for word in tweet.content.split():
                    sid_obj_word = SentimentIntensityAnalyzer()
                    sentiment_dict_word = sid_obj_word.polarity_scores(word)
                    if sentiment_dict_word['compound'] >= 0.05:
                        pos_count_tweet += 1
                        total_count_tweet += 1
                    elif sentiment_dict_word['compound'] <= -0.05:
                        neg_count_tweet += 1
                        total_count_tweet += 1
                    else:
                        total_count_tweet += 1

                #print(pos_count_tweet, neg_count_tweet, total_count_tweet)
                ####################### Polarity ###########################################
                #polarity = functions.get_tweet_sentiment(tweet.content) textblob
                sid_obj = SentimentIntensityAnalyzer()
 

                sentiment_dict = sid_obj.polarity_scores(tweet.content)
     

                polarity = sentiment_dict['compound']
    
                if polarity >= 0.05:
                    tweets_list.append([tweet.username, tweet.date, tweet.content, pos_count_tweet, neg_count_tweet, total_count_tweet, polarity, "Positivo"])
                
                elif polarity <= -0.05:
                    tweets_list.append([tweet.username, tweet.date, tweet.content, pos_count_tweet, neg_count_tweet, total_count_tweet, polarity, "Negativo"])
                else:
                    
                    tweets_list.append([tweet.username, tweet.date, tweet.content, pos_count_tweet, neg_count_tweet, total_count_tweet, polarity, "Neutro"])


get_tweets()

# Creating a dataframe from the tweets list above 
tweets_df1 = pd.DataFrame(tweets_list, columns=['User','Datetime', 'Text', 'Pos_Count', 'Neg_Count', 'Total of word', 'Polarity_count', 'Polarity'])
tweets_df1.to_csv('srcrape_tweets_vader_neutro_clean_12_02_28_02.csv')
print(tweets_df1)





## Parte 3 - Recolha das informações sobre o utilizador

In [ ]:
#################### TWEEPY ############################
import tweepy

import pandas as pd
import snscrape.modules.twitter as sntwitter
from datetime import date

#########################################################
consumer_key = "XXXXXXX"
consumer_secret = "XXXXXXXX"
access_token = "XXXXXXXXX"
access_token_secret = "XXXXXXXXXX"

################## LIST OF USERS ########################
# extract a column from cvs file and convert to string

data_12_02 = pd.read_csv('srcrape_tweets_vader_neutro_clean_12_02_28_02.csv')
data_12_01 = pd.read_csv('srcrape_tweets_vader_neutro_clean_12_01_27_01.csv')
info_users = pd.read_csv('srcrape_tweets_user_information_2.csv')
user_info_list = info_users['User'].tolist()
#print(user_info_list)
list_data_1 = data_12_01.to_numpy().tolist()
list_data_2 = data_12_01.to_numpy().tolist()
#converting column data to list
user_name_list_not_clean = data_12_02['User'].tolist()
#print(user_name_list_not_clean)
user_name_list_not_clean.extend(data_12_01['User'].tolist())


user_list_cleanm = []
user_name_list_clean  = list(dict.fromkeys(user_name_list_not_clean))
print(len(user_name_list_clean))
for name in user_name_list_clean:
    if name  not in user_info_list:
       user_list_cleanm.append(name)

users_list_info = []
print(f"Number of total users: {len(user_name_list_clean)}")
#########################################################
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  
# set access to user's access key and access secret 
auth.set_access_token(access_token, access_token_secret)
  
# calling the api 
todays_date = date.today()
api = tweepy.API(auth)
################## USER DATA#############################
count = 0
for screen_name in user_list_cleanm:
    user_tweets_i = []
    # fetching the user
    
    if api.get_user(screen_name=screen_name):
        user = api.get_user(screen_name=screen_name)
    
    
    # fetching the user
        
        #Number of Followers:
        # fetching the followers_count
        followers_count = user.followers_count

        #2) Number of Tweets:
        statuses_count = user.statuses_count 

        #3) Number of retweets and favorites
        
        #5) Number of Public Lists:
        listed_count = user.listed_count

        #6) Actions to Tweets Ratio: 
        #retweet_count = tweet.retweet_count
        #favorite_count = tweet.favorite_count
        #favourites_count = user.favourites_count
        
        #11) New Tweets:  dentro do periodo indicado

        # retweets += tweet.retweet_count
            #favorites += tweet.favorite_count
        #action_ratio = (retweet_count + favorite_count) / statuses_count

        #7) Followers to Friend Ratio: 
        friends_count = user.friends_count
        print(f"Friends count: {friends_count} for {screen_name}")
        if friends_count > 0:
            followers_friend_ratio = followers_count / friends_count
        else:
            followers_friend_ratio = 0
    
        #8) Age of Account: 
        
        created_at = user.created_at
        date = str(created_at).split()[0]
        year = date.split("-")[0]

        age_of_account_years = int(todays_date.year) - int(year)
    
        #9) New Followers:

        #10) New Favorites: 

        #11) New Tweets:  dentro do periodo indicado



    # new_tweets_count, new_favorites = get_tweets(api, screen_name)

    
        for tweet in list_data_1:
            if screen_name == tweet[1]:
                user_tweets_i.append(tweet)
        for tweet in list_data_2:
            if screen_name == tweet[1]:
                user_tweets_i.append(tweet)


        users_list_info.append([screen_name, followers_count, statuses_count,  listed_count,  followers_friend_ratio, age_of_account_years, user_tweets_i])

        if count < 400:
            count += 1
        else:
            break
################# CREATE A DATAFRAME ####################

tweets_df1 = pd.DataFrame(users_list_info, columns=['User','Followers', 'Num_Tweets', 'Num_public_lists', 'Followers_friend', 'Age_account'])
tweets_df1.to_csv('srcrape_tweets_user_information_tweets.csv')

##########################################################



## Parte 4 - Aplicação do algoritmo genético

In [47]:
################# Converter de csv para dataframe ########################

import pandas as pd 
#tweets_df1 = pd.DataFrame(users_list_info, columns=['User','Followers', 'Num_Tweets', 'Num_public_lists', 'Followers_friend', 'Age_account'])
# making dataframe 
df = pd.read_csv("srcrape_tweets_user_information_2.csv") 
############################### SORT ##############################
########################## By Followers ###########################

df_followers = df.sort_values(by=['Followers'], ascending=False)
rank_1 = df_followers.iloc[:40]
rank_2 = df_followers.iloc[40:80]
rank_3 = df_followers.iloc[80:120]
rank_4 = df_followers.iloc[120:160]
rank_5 = df_followers.iloc[160:200]
rank_6 = df_followers.iloc[200:240]
rank_7 = df_followers.iloc[240:280]
rank_8 = df_followers.iloc[280:320]
rank_9 = df_followers.iloc[320:340]
rank_10 = df_followers.iloc[340:380]
rank_11 = df_followers.iloc[380:420]
rank_12 = df_followers.iloc[420:460]
rank_13 = df_followers.iloc[460:500]
rank_14 = df_followers.iloc[540:580]
rank_15 = df_followers.iloc[580:630]
rank_16 = df_followers.iloc[620:660]
rank_17 = df_followers.iloc[660:700]
rank_18 = df_followers.iloc[700:740]
rank_19 = df_followers.iloc[740:780]
rank_20 = df_followers.iloc[780:800]

###################################################################

##########################3 By Num Tweets #########################

df_num_tweets = df.sort_values(by=['Num_Tweets'], ascending=False)
rank_1 = df_num_tweets.iloc[:40]
rank_2 = df_num_tweets.iloc[40:80]
rank_3 = df_num_tweets.iloc[80:120]
rank_4 = df_num_tweets.iloc[120:160]
rank_5 = df_num_tweets.iloc[160:200]
rank_6 = df_num_tweets.iloc[200:240]
rank_7 = df_num_tweets.iloc[240:280]
rank_8 = df_num_tweets.iloc[280:320]
rank_9 = df_num_tweets.iloc[320:340]
rank_10 = df_num_tweets.iloc[340:380]
rank_11 = df_num_tweets.iloc[380:420]
rank_12 = df_num_tweets.iloc[420:460]
rank_13 = df_num_tweets.iloc[460:500]
rank_14 = df_num_tweets.iloc[540:580]
rank_15 = df_num_tweets.iloc[580:630]
rank_16 = df_num_tweets.iloc[620:660]
rank_17 = df_num_tweets.iloc[660:700]
rank_18 = df_num_tweets.iloc[700:740]
rank_19 = df_num_tweets.iloc[740:780]
rank_20 = df_num_tweets.iloc[780:800]


###################################################################

##########################3 By Num_public_lists ###################
df_num_public_lists = df.sort_values(by=['Num_public_lists'], ascending=False)

rank_1 = df_num_public_lists.iloc[:40]
rank_2 = df_num_public_lists.iloc[40:80]
rank_3 = df_num_public_lists.iloc[80:120]
rank_4 = df_num_public_lists.iloc[120:160]
rank_5 = df_num_public_lists.iloc[160:200]
rank_6 = df_num_public_lists.iloc[200:240]
rank_7 = df_num_public_lists.iloc[240:280]
rank_8 = df_num_public_lists.iloc[280:320]
rank_9 = df_num_public_lists.iloc[320:340]
rank_10 = df_num_public_lists.iloc[340:380]
rank_11 = df_num_public_lists.iloc[380:420]
rank_12 = df_num_public_lists.iloc[420:460]
rank_13 = df_num_public_lists.iloc[460:500]
rank_14 = df_num_public_lists.iloc[540:580]
rank_15 = df_num_public_lists.iloc[580:630]
rank_16 = df_num_public_lists.iloc[620:660]
rank_17 = df_num_public_lists.iloc[660:700]
rank_18 = df_num_public_lists.iloc[700:740]
rank_19 = df_num_public_lists.iloc[740:780]
rank_20 = df_num_public_lists.iloc[780:800]

###################################################################

##########################3 By Followers_friend ##################
df_racio = df.sort_values(by=['Followers_friend'], ascending=False)
#print(df_racio)

rank_1 = df_racio.iloc[:40]
rank_2 = df_racio.iloc[40:80]
rank_3 = df_racio.iloc[80:120]
rank_4 = df_racio.iloc[120:160]
rank_5 = df_racio.iloc[160:200]
rank_6 = df_racio.iloc[200:240]
rank_7 = df_racio.iloc[240:280]
rank_8 = df_racio.iloc[280:320]
rank_9 = df_racio.iloc[320:340]
rank_10 = df_racio.iloc[340:380]
rank_11 = df_racio.iloc[380:420]
rank_12 = df_racio.iloc[420:460]
rank_13 = df_racio.iloc[460:500]
rank_14 = df_racio.iloc[540:580]
rank_15 = df_racio.iloc[580:630]
rank_16 = df_racio.iloc[620:660]
rank_17 = df_racio.iloc[660:700]
rank_18 = df_racio.iloc[700:740]
rank_19 = df_racio.iloc[740:780]
rank_20 = df_racio.iloc[780:800]


###################################################################

##########################3 By Age_account #########################
df_age = df.sort_values(by=['Age_account'], ascending=False)

rank_1 = df_age.iloc[:40]
rank_2 = df_age.iloc[40:80]
rank_3 = df_age.iloc[80:120]
rank_4 = df_age.iloc[120:160]
rank_5 = df_age.iloc[160:200]
rank_6 = df_age.iloc[200:240]
rank_7 = df_age.iloc[240:280]
rank_8 = df_age.iloc[280:320]
rank_9 = df_age.iloc[320:340]
rank_10 = df_age.iloc[340:380]
rank_11 = df_age.iloc[380:420]
rank_12 = df_age.iloc[420:460]
rank_13 = df_age.iloc[460:500]
rank_14 = df_age.iloc[540:580]
rank_15 = df_age.iloc[580:630]
rank_16 = df_age.iloc[620:660]
rank_17 = df_age.iloc[660:700]
rank_18 = df_age.iloc[700:740]
rank_19 = df_age.iloc[740:780]
rank_20 = df_age.iloc[780:800]


################################## Evaluation Function ##############################################














     Unnamed: 0            User  Followers  Num_Tweets  Num_public_lists  \
568         568     JonahLupton     556176      147259              5385   
615         615  charliebilello     379176       24098              7011   
360         360       ripster47     283262       43571              6051   
657         657        BigCheds     241363      161446              3662   
12           12    MarketRebels     232545       46295              2057   
..          ...             ...        ...         ...               ...   
227         227      SamiFathi_      10677        8905                93   
310         310  Titan_Traders_      10645       19599               117   
374         374        iramneek      10071        9975               235   
736         736  everytimeicash       9953       43617               237   
32           32       dylan522p       9894        3596               275   

     Followers_friend  Age_account  
568        289.675000           14  
615       315

### Parte 5 - Yahoo!Finance

In [7]:
from pandas import *
import yfinance as yf
data = yf.download("AAPL", start='2022-01-12', end='2022-02-12')
print(data.columns)

############################ DATE ######################

data.loc["2022-01-12":"2022-02-12"]
data.to_excel("date.xlsm")
date_df = pd.read_excel('date.xlsm') # can also index sheet by name or fetch all sheets
mylist = date_df['Date'].tolist()

data_12_02 = pd.read_csv('srcrape_tweets_vader_neutro_clean_12_02_28_02.csv')
data_12_01 = pd.read_csv('srcrape_tweets_vader_neutro_clean_12_01_27_01.csv')
info_users = pd.read_csv('srcrape_tweets_user_information_2.csv')
user_info_list = info_users['User'].tolist()

user_tweet_user = []
for user in user_info_list:
    dt_user = data_12_02.loc[data_12_02['User'] == user]
    user_tweet_user.append([user, dt_user])

#########################################################


list_gain = []
list_values = data.values.tolist()


gain = 0
#print(list_values)
for i in range(len(list_values)):
    if i == 0:
        gain = 0
    else:
        acp_yesterday = list_values[i-1][5]
        acp_today = list_values[i][5]

        gain = ((acp_today - acp_yesterday) / (acp_yesterday) )* 100
        list_gain.append([str(mylist[i]), acp_yesterday, acp_today, gain])



print(list_gain)

[*********************100%***********************]  1 of 1 completed
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')
[['2022-01-13 00:00:00', 74805200.0, 84505800.0, 12.967815071679508], ['2022-01-14 00:00:00', 84505800.0, 80440800.0, -4.810320711714462], ['2022-01-18 00:00:00', 80440800.0, 90956700.0, 13.07284363158994], ['2022-01-19 00:00:00', 90956700.0, 94815000.0, 4.241908512511997], ['2022-01-20 00:00:00', 94815000.0, 91420500.0, -3.580129726309128], ['2022-01-21 00:00:00', 91420500.0, 122848900.0, 34.37784741934249], ['2022-01-24 00:00:00', 122848900.0, 162294600.0, 32.10911941417465], ['2022-01-25 00:00:00', 162294600.0, 115798400.0, -28.64925881699083], ['2022-01-26 00:00:00', 115798400.0, 108275300.0, -6.496721889076188], ['2022-01-27 00:00:00', 108275300.0, 121954600.0, 12.633813990817849], ['2022-01-28 00:00:00', 121954600.0, 179935700.0, 47.543184102936664], ['2022-01-31 00:00:00', 179935700.0, 115541600.0, -35.787284013122466], ['2022-02-01 